In [ ]:
!git clone https://github.com/neetcode-gh/leetcode.git

In [ ]:
!pip install libcst
!pip install bertopic
!pip install openai

In [3]:
import os
import pandas as pd
import libcst as cst

def parse_python_files(folder_path: str):
    """
    Loop through a folder, for every Python file parse the file using libcst,
    and then store in a pandas dataframe with columns (filename, code, libcst tree).

    Args:
    - folder_path (str): Path to the directory containing Python files.

    Returns:
    - pd.DataFrame: DataFrame with columns (filename, code, libcst tree).
    """

    # Initializing lists to hold data
    filenames = []
    codes = []
    libcst_trees = []

    # Loop through each file in the specified folder
    for root, _, files in os.walk(folder_path):
        for file in files:
            # Check if the file is a Python file
            if file.endswith('.py'):
                file_path = os.path.join(root, file)

                # Reading the file content
                with open(file_path, 'r', encoding='utf-8') as f:
                    code = f.read()

                # Parsing the code using libcst
                try:
                    tree = cst.parse_module(code)
                    # Appending data to the lists
                    filenames.append(file_path.split('/')[-1])
                    codes.append(code)
                    libcst_trees.append(tree)
                except Exception as e:
                    print(f"Error parsing {file_path}: {e}")

    # Creating a pandas DataFrame
    df = pd.DataFrame({
        'filename': filenames,
        'code': codes,
        'libcst_tree': libcst_trees
    })

    return df


In [5]:
df = parse_python_files("/content/leetcode/python")

In [6]:
df

,filename,code,libcst_tree
0,0703-kth-largest-element-in-a-stream.py,"class KthLargest:\n def __init__(self, k: i...",Module(\n body=[\n ClassDef(\n ...
1,2390-removing-stars-from-a-string.py,# https://leetcode.com/problems/removing-stars...,Module(\n body=[\n ClassDef(\n ...
2,0355-design-twitter.py,class Twitter:\n def __init__(self):\n ...,Module(\n body=[\n ClassDef(\n ...
3,0105-construct-binary-tree-from-preorder-and-i...,"class Solution:\n def buildTree(self, preor...",Module(\n body=[\n ClassDef(\n ...
4,0901-online-stock-span.py,class StockSpanner:\n def __init__(self):\n...,Module(\n body=[\n ClassDef(\n ...
...,...,...,...
328,0740-delete-and-earn.py,# House Robber Style\n# Time Complexity O(n)\n...,Module(\n body=[\n ClassDef(\n ...
329,0146-lru-cache.py,"class Node:\n def __init__(self, key, val):...",Module(\n body=[\n ClassDef(\n ...
330,0474-ones-and-zeroes.py,"class Solution:\n def findMaxForm(self, str...",Module(\n body=[\n ClassDef(\n ...
331,0088-merge-sorted-array.py,"class Solution:\n def merge(self, nums1: Li...",Module(\n body=[\n ClassDef(\n ...


## Count Data Structures

In [11]:
import libcst as cst

class DataStructureCounter(cst.CSTVisitor):
    """
    Visitor class to count the number of data structures used in Python code.
    """

    def __init__(self):
        super().__init__()
        self.list_count = 0
        self.tuple_count = 0
        self.set_count = 0
        self.dict_count = 0

    def visit_List(self, node: cst.List) -> None:
        """
        Visit all list nodes.
        """
        self.list_count += 1

    def visit_Tuple(self, node: cst.Tuple) -> None:
        """
        Visit all tuple nodes.
        """
        self.tuple_count += 1

    def visit_Set(self, node: cst.Set) -> None:
        """
        Visit all set nodes.
        """
        self.set_count += 1

    def visit_Dict(self, node: cst.Dict) -> None:
        """
        Visit all dictionary nodes.
        """
        self.dict_count += 1


def count_data_structures_in_code(code: str) -> dict:
    """
    Count the number of data structures used in the given code.

    Args:
    - code (str): Python code.

    Returns:
    - dict: Dictionary containing counts of different data structures.
    """
    module = cst.parse_module(code)
    visitor = DataStructureCounter()
    module.visit(visitor)

    return {
        "list_count": visitor.list_count,
        "tuple_count": visitor.tuple_count,
        "set_count": visitor.set_count,
        "dict_count": visitor.dict_count
    }
def count_data_structures(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the given pandas DataFrame and add columns for counts of different
    data structures in the code column.

    Args:
    - df (pd.DataFrame): DataFrame with a 'code' column containing Python code.

    Returns:
    - pd.DataFrame: Updated DataFrame with additional columns for data structure counts.
    """

    # Apply the count_data_structures_in_code function to the 'code' column
    results = df['code'].apply(count_data_structures_in_code)

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results.tolist())

    # Concatenate the results to the original DataFrame
    df = pd.concat([df, results_df], axis=1)

    return df


In [12]:
df = count_data_structures(df)

In [13]:
df

,filename,code,libcst_tree,list_count,tuple_count,set_count,dict_count
0,0097-interleaving-string.py,"class Solution:\n def isInterleave(self, s1...",Module(\n body=[\n ClassDef(\n ...,1,0,0,0
1,0013-roman-to-integer.py,"class Solution:\n def romanToInt(self, s: s...",Module(\n body=[\n ClassDef(\n ...,0,0,0,1
2,0509-fibonacci-number.py,class Solution:\n Memo = {}\n\n def fib(...,Module(\n body=[\n ClassDef(\n ...,0,0,0,1
3,0211-design-add-and-search-words-data-structur...,class TrieNode:\n def __init__(self):\n ...,Module(\n body=[\n ClassDef(\n ...,0,0,0,1
4,1461-check-if-a-string-contains-all-binary-cod...,"class Solution:\n def hasAllCodes(self, s: ...",Module(\n body=[\n ClassDef(\n ...,0,0,0,0
...,...,...,...,...,...,...,...
328,1822-sign-of-the-product-of-an-array.py,"class Solution:\n def arraySign(self, nums:...",Module(\n body=[\n ClassDef(\n ...,0,0,0,0
329,0268-missing-number.py,"class Solution:\n def missingNumber(self, n...",Module(\n body=[\n ClassDef(\n ...,0,0,0,0
330,0516-longest-palindromic-subsequence.py,# Time: O(n^2) Space: O(n^2) - For all three s...,Module(\n body=[\n ClassDef(\n ...,2,7,0,1
331,0155-min-stack.py,class MinStack:\n def __init__(self):\n ...,Module(\n body=[\n ClassDef(\n ...,2,0,0,0


In [14]:
def print_code_from_row(df: pd.DataFrame, row_index: int) -> None:
    """
    Print the contents of the 'code' column for the given row index.

    Args:
    - df (pd.DataFrame): DataFrame containing the 'code' column.
    - row_index (int): The index of the row to print the code from.

    Returns:
    - None: The function prints the code and doesn't return anything.
    """
    try:
        code_content = df.loc[row_index, 'code']
        print(code_content)
    except KeyError:
        print(f"Row index {row_index} not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred: {e}")

print_code_from_row(df, 330)

# Time: O(n^2) Space: O(n^2) - For all three solutions
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:   
        # Dynamic Programming
        dp = [ [0] * (len(s) + 1) for i in range(len(s) + 1)]
        res = 0
        
        for i in range(len(s)):
            for j in range(len(s) - 1, i - 1, -1):
                if s[i] == s[j]:
                    dp[i][j] = 1 if i == j else 2
                    if i - 1 >= 0:
                        dp[i][j] += dp[i - 1][j + 1]
                else:
                    dp[i][j] = dp[i][j + 1]
                    if i - 1 >= 0:
                        dp[i][j] = max(dp[i][j], dp[i - 1][j])
                res = max(res, dp[i][j])
        return res


        # Memoization
        cache = {}

        def dfs(i, j):
            if i < 0 or j == len(s):
                return 0
            if (i, j) in cache:
                return cache[(i, j)]

            if s[i] == s[j]:
                length = 1 if i == j else 2
     

## Detect Recursion

In [25]:
df = parse_python_files("/content/leetcode/python")


In [22]:
import libcst as cst

class RecursiveFunctionDetector(cst.CSTVisitor):
    def __init__(self):
        self.current_function_name = None
        self.recursive_functions = set()


    def visit_FunctionDef(self, node: cst.FunctionDef) -> None:
        # Set the current function name
        self.current_function_name = node.name.value

    def leave_FunctionDef(self, node: cst.FunctionDef) -> None:
        # Reset the current function name after leaving the function
        self.current_function_name = None

    def visit_Call(self, node: cst.Call) -> None:
        if self.current_function_name is not None:
            # Check if the called function has the same name as the current function
            if isinstance(node.func, cst.Name) and node.func.value == self.current_function_name:
                self.recursive_functions.add(self.current_function_name)

def detect_recursive_functions(code: str) -> set:
    """
    Detect direct recursive functions in the given code.

    Args:
    - code (str): Python code as a string.

    Returns:
    - set: Set of function names that are recursive.
    """
    module = cst.parse_module(code)
    uses_recursion = False
    visitor = RecursiveFunctionDetector()
    module.visit(visitor)
    if len(list(visitor.recursive_functions)) > 0:
        uses_recursion = True
    return {
        "uses_recursion": uses_recursion,
        "recursive_functions":visitor.recursive_functions
        }

def label_recursive_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the given pandas DataFrame and add columns for counts of different
    data structures in the code column.

    Args:
    - df (pd.DataFrame): DataFrame with a 'code' column containing Python code.

    Returns:
    - pd.DataFrame: Updated DataFrame with additional columns for data structure counts.
    """

    # Apply the count_data_structures_in_code function to the 'code' column
    results = df['code'].apply(detect_recursive_functions)

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results.tolist())

    # Concatenate the results to the original DataFrame
    df = pd.concat([df, results_df], axis=1)

    return df

In [26]:
df = label_recursive_rows(df)



In [27]:
df[df['uses_recursion'] == True]

,filename,code,libcst_tree,uses_recursion,recursive_functions
3,0211-design-add-and-search-words-data-structur...,class TrieNode:\n def __init__(self):\n ...,Module(\n body=[\n ClassDef(\n ...,True,{dfs}
10,0110-balanced-binary-tree.py,# Definition for a binary tree node.\n# class ...,Module(\n body=[\n ClassDef(\n ...,True,{dfs}
17,0078-subsets.py,"class Solution:\n def subsets(self, nums: L...",Module(\n body=[\n ClassDef(\n ...,True,{dfs}
25,2002-maximum-product-of-the-length-of-two-pali...,"""""""\nTime Complexity: O(2^N)\nSpace Complexity...",Module(\n body=[\n SimpleStatementLi...,True,{dp}
38,0018-4sum.py,"class Solution:\n def fourSum(self, nums, t...",Module(\n body=[\n ClassDef(\n ...,True,{findNsum}
...,...,...,...,...,...
286,0698-partition-to-k-equal-sum-subsets.py,class Solution(object):\n def canPartitionK...,Module(\n body=[\n ClassDef(\n ...,True,{backtrack}
291,0063-unique-paths-ii.py,class Solution:\n def uniquePathsWithObstac...,Module(\n body=[\n ClassDef(\n ...,True,{dfs}
297,0039-combination-sum.py,"class Solution:\n def combinationSum(self, ...",Module(\n body=[\n ClassDef(\n ...,True,{dfs}
302,0077-combinations.py,"class Solution:\n def combine(self, n: int,...",Module(\n body=[\n ClassDef(\n ...,True,{helper}


In [28]:
print_code_from_row(df[df['uses_recursion'] == True], 17)


class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        res = []

        subset = []

        def dfs(i):
            if i >= len(nums):
                res.append(subset.copy())
                return
            # decision to include nums[i]
            subset.append(nums[i])
            dfs(i + 1)
            # decision NOT to include nums[i]
            subset.pop()
            dfs(i + 1)

        dfs(0)
        return res



## Topic modeling and Clustering

In [10]:
from google.colab import userdata


In [69]:
DEFAULT_PROMPT = """
This is a list of code where each collection of code describe a topic. After each collection of code, the name of the topic they represent is mentioned as a short-highly-descriptive title
Mention the different patterns and any other specific descriptors that the examples share
Arrays & Hashing
Two Pointers
Sliding Window
Stack
Binary Search
Linked List
Trees
Tries
Heap / Priority Queue
Backtracking
Graphs
Advanced Graphs
Divide and Conquer
1-D Dynamic Programming
2-D Dynamic Programming
Greedy
Intervals
Math & Geometry
Bit Manipulation

Topic:
Sample texts from this topic:
[DOCUMENTS]
Keywords: [KEYWORDS]
Topic name:"""


In [70]:
import openai
from bertopic import BERTopic
from bertopic.representation import OpenAI
from umap import UMAP
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine')


# Fine-tune topic representations with GPT
openai.api_key = userdata.get('openai_key')
representation_model = OpenAI(model="gpt-4", chat=True,exponential_backoff=True)



In [71]:
representation_model.default_prompt_ = DEFAULT_PROMPT

In [72]:
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, representation_model=representation_model)

In [73]:
docs = df['code'].to_list()
topics, probs = topic_model.fit_transform(docs)

In [74]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,40,-1_Python Programming Solutions,[Python Programming Solutions],[class Solution:\n def findDisappearedNumbe...
1,0,50,0_Dynamic Programming in String Manipulation a...,[Dynamic Programming in String Manipulation an...,"[class Solution:\n def findMaxForm(self, st..."
2,1,26,1_Linked List Manipulation Methods in Python,[Linked List Manipulation Methods in Python],"[class Solution:\n def pairSum(self, head: ..."
3,2,23,2_Programming - Data Structures and Algorithms,[Programming - Data Structures and Algorithms],[class Solution:\n def reorganizeString(sel...
4,3,23,3_Binary Tree Node Operations,[Binary Tree Node Operations],[# Definition for a binary tree node.\n# class...
5,4,23,4_Coding Solutions for Array and List Manipula...,[Coding Solutions for Array and List Manipulat...,[class Solution:\n def majorityElement(self...
6,5,21,5_Programming Solutions for Various Algorithms,[Programming Solutions for Various Algorithms],"[class Solution:\n def minSubArrayLen(self,..."
7,6,20,6_Binary Search Algorithms in Python,[Binary Search Algorithms in Python],[class Solution:\n def findPeakElement(self...
8,7,15,7_Python Coding Solutions for Array and String...,[Python Coding Solutions for Array and String ...,"[class Solution:\n def removeElements(self,..."
9,8,13,8_Manipulation and Analysis of 2D Grids in Python,[Manipulation and Analysis of 2D Grids in Python],"[class Solution:\n def rotate(self, matrix:..."


In [75]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,"class KthLargest:\n def __init__(self, k: i...",2,2_Programming - Data Structures and Algorithms,[Programming - Data Structures and Algorithms],[class Solution:\n def reorganizeString(sel...,Programming - Data Structures and Algorithms,1.000000,False
1,# https://leetcode.com/problems/removing-stars...,7,7_Python Coding Solutions for Array and String...,[Python Coding Solutions for Array and String ...,"[class Solution:\n def removeElements(self,...",Python Coding Solutions for Array and String M...,1.000000,False
2,class Twitter:\n def __init__(self):\n ...,2,2_Programming - Data Structures and Algorithms,[Programming - Data Structures and Algorithms],[class Solution:\n def reorganizeString(sel...,Programming - Data Structures and Algorithms,0.820472,False
3,"class Solution:\n def buildTree(self, preor...",3,3_Binary Tree Node Operations,[Binary Tree Node Operations],[# Definition for a binary tree node.\n# class...,Binary Tree Node Operations,0.474709,False
4,class StockSpanner:\n def __init__(self):\n...,16,16_Stack and Queue Operations in Python Classes,[Stack and Queue Operations in Python Classes],[class Solution(object):\n def validateStac...,Stack and Queue Operations in Python Classes,0.941423,True
...,...,...,...,...,...,...,...,...
328,# House Robber Style\n# Time Complexity O(n)\n...,7,7_Python Coding Solutions for Array and String...,[Python Coding Solutions for Array and String ...,"[class Solution:\n def removeElements(self,...",Python Coding Solutions for Array and String M...,1.000000,False
329,"class Node:\n def __init__(self, key, val):...",1,1_Linked List Manipulation Methods in Python,[Linked List Manipulation Methods in Python],"[class Solution:\n def pairSum(self, head: ...",Linked List Manipulation Methods in Python,1.000000,False
330,"class Solution:\n def findMaxForm(self, str...",0,0_Dynamic Programming in String Manipulation a...,[Dynamic Programming in String Manipulation an...,"[class Solution:\n def findMaxForm(self, st...",Dynamic Programming in String Manipulation and...,1.000000,True
331,"class Solution:\n def merge(self, nums1: Li...",14,14_Merging and Sorting Lists and Linked Lists ...,[Merging and Sorting Lists and Linked Lists in...,[# Definition for singly-linked list.\n# class...,Merging and Sorting Lists and Linked Lists in ...,1.000000,False


In [78]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

100%|██████████| 17/17 [00:39<00:00,  2.31s/it]


gpt-4

In [79]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

.
├─Binary Tree Operations and Solutions
│    ├─Linked List Manipulation in Python
│    │    ├─■──Stack and Queue Operations in Python Classes ── Topic: 16
│    │    └─Linked List Manipulation Methods in Python
│    │         ├─■──Linked List Manipulation Methods in Python ── Topic: 1
│    │         └─■──Merging and Sorting Lists and Linked Lists in Python ── Topic: 14
│    └─Binary Tree Operations
│         ├─■──Binary Tree Node Operations ── Topic: 3
│         └─■──Binary Tree Operations using Depth-First Search ── Topic: 10
└─Python Programming - Algorithm Solutions
     ├─Programming Solutions Using Depth-First Search (DFS)
     │    ├─■──Word Processing & Search Algorithms in Python ── Topic: 9
     │    └─Grid Processing Algorithms in Python
     │         ├─■──Depth-First Search in Grids and Matrices ── Topic: 11
     │         └─■──Manipulation and Analysis of 2D Grids in Python ── Topic: 8
     └─Python Programming - Solution Classes for Various Algorithms
          ├─Python P

gpt3.5-turbo

In [61]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

.
├─Binary Tree Operations
│    ├─Linked List Operations
│    │    ├─Stack Operations in Python
│    │    │    ├─■──Programming Topics with Various Classes and Methods ── Topic: 19
│    │    │    └─Stack Manipulation
│    │    │         ├─■──Stack Manipulation ── Topic: 7
│    │    │         └─■──Stack-based String Operations ── Topic: 17
│    │    └─Linked List Operations in Python
│    │         ├─■──Linked List Operations ── Topic: 0
│    │         └─■──Merge and Sort Linked Lists ── Topic: 16
│    └─Binary Tree Operations
│         ├─■──Binary Tree Operations ── Topic: 2
│         └─■──Binary Tree Topic ── Topic: 13
└─Python coding
     ├─Data Structures and Algorithms
     │    ├─Python coding - board games and graph traversal
     │    │    ├─■──Python grid manipulation methods ── Topic: 10
     │    │    └─■──Python coding - Trie, Word Matching ── Topic: 9
     │    └─Dynamic Programming
     │         ├─■──Python String and List Manipulation Methods ── Topic: 5
     │         └